In [8]:
#svm
import menpo.io as mio
import os
from sklearn import svm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
import math
import Models
from Models import ChangeVector, ImageData
path_to_svm_training_database = '/Programing/GR/Code/CK+/aam-images/**/**/**/*'
import pickle


def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def process(image, crop_proportion=0.2, max_diagonal=400):
    if image.n_channels == 3:
        image = image.as_greyscale()
    image = image.crop_to_landmarks_proportion(crop_proportion)
    d = image.diagonal()
    if d > max_diagonal:
        image = image.rescale(float(max_diagonal) / d)
    return image

#process changingVector, reduce dimension from 2x68 to 1x68, by process PCA
def pca(changeVector):
    X = np.array(changeVector)
    pca_model = PCA(n_components=1)
    return pca_model.fit_transform(X)

#chuan hoa theo max tung landmark
def landmark_normalize(landmark):
    vector_max = 0
    for p in landmark:
        if(math.sqrt(float(p[0])*float(p[0]) + float(p[1])*float(p[1])) > vector_max):
            vector_max = math.sqrt(p[0]*p[0] + p[1]*p[1])
    for p in landmark:
        p[0] = p[0]/float(vector_max)
        p[1] = p[1]/float(vector_max)
    return landmark

#chuan hoa mang 68 diem landmark cua perk state theo neutral state
def normalize_perk_landmark(landmark_perk, landmark_neutral):
    neutral_center = landmark_neutral[30]
    perk_center = landmark_perk[30]
#     for lm in landmark_perk:
    #move perk landmark to match center of neutral landmark
    move_vector = [neutral_center[0] - perk_center[0], neutral_center[1] - perk_center[1]]
    for lm in landmark_perk:
        lm[0] += move_vector[0]
        lm[1] += move_vector[1]
    
    #scale the size of detected perk landmark based on neutrol landmark 27 to 30 (the nose)
    scale_neutral = [landmark_neutral[30][0] - landmark_neutral[27][0], landmark_neutral[30][1] - landmark_neutral[27][1]]
    scale_perk = [landmark_perk[30][0] - landmark_perk[27][0], landmark_perk[30][1] - landmark_perk[27][1]]
    ratio = math.sqrt(scale_neutral[0]*scale_neutral[0] + scale_neutral[1]*scale_neutral[1])/math.sqrt(scale_perk[0]*scale_perk[0]+scale_perk[1]*scale_perk[1]) 
    for lm in landmark_perk:
        lm[0] = (perk_center[0] - lm[0]) * (1 - ratio) + lm[0]
        lm[1] = (perk_center[1] - lm[1]) * (1 - ratio) + lm[1]
        
    return landmark_perk

#import images
training_images = mio.import_images(path_to_svm_training_database, verbose=True)
training_images = training_images.map(process)

path_to_facs = '/Programing/GR/Code/CK+/FACS/'
path_to_emotions = '/Programing/GR/Code/CK+/Emotion/'

#create training data

#check if example is labeled
labeled_subject = []
emotion_subject = os.listdir(path_to_emotions)
for subject in emotion_subject:
    session = os.listdir(path_to_emotions + "/" + subject)
    for s in session:
        labeled_subject.append(subject + "-" + s)

count = 0;
svm_training_data = []
while(count < len(training_images)):
    file_path = str(training_images[count].path).split("\\")
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
    landmark = []
    landmark_neutral = training_images[count].landmarks['PTS'].lms.points
    landmark_perk = training_images[count + 1].landmarks['PTS'].lms.points
    for i in range(0,68):
        landmark.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])

#     svm_training_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   

    svm_training_data.append(ImageData(data_facs, landmark, gt_emotion))
    count = count + 2
    

facs = []
#create facs array
for data in svm_training_data:
    for facs_code in data.facs:
        if((int(facs_code)) not in facs):
            facs.append(int(facs_code))
facs.sort()

#create testing data
svm_testing_data = []
path_to_svm_testing_database = "/Programing/GR/Code/CK+/test-aam-images/**/**/**/*"
testing_images = mio.import_images(path_to_svm_testing_database, verbose=True)
testing_images = testing_images.map(process)

count = 0;
while(count < len(testing_images)):
    file_path = str(testing_images[count].path).split("\\")
#     print(file_path)
    facs_path = path_to_facs + file_path[6] + '/' + file_path[7]
    gt_emotion = -1
    
    #get emotion label from Emotion Folder
    emotion_checker = file_path[6] + "-" + file_path[7]
    if(emotion_checker in  labeled_subject):
        emotion_path = path_to_emotions + file_path[6] + '/' + file_path[7]
        if(len(os.listdir(emotion_path)) != 0):
            emotion_path = emotion_path + '/' + os.listdir(emotion_path)[0]
            fi = open(emotion_path)
            for line in fi:
                if(line.split()):
                    gt_emotion = int(float(line.split()[0]))
#                 print(emotion_path + ":" + str(gt_emotion))
            fi.close()
    
    
    #get facs from FACS folder
    facs_path = facs_path + '/' + os.listdir(facs_path)[0]
    fi = open(facs_path, 'r')
    data_facs = {}
    tmp = []
    for line in fi: # read rest of lines
        if(line.split()):
            tmp.append(line.split())
    for f in tmp:
        data_facs[str(int(float(f[0])))] = int(float(f[1]))
    fi.close()
    
    landmark = []
    landmark_neutral = testing_images[count].landmarks['PTS'].lms.points
    landmark_perk = testing_images[count + 1].landmarks['PTS'].lms.points
    landmark_perk = normalize_perk_landmark(landmark_neutral, landmark_perk)
    for i in range(0,68):
        landmark.append([landmark_perk[i][0] - landmark_neutral[i][0], landmark_perk[i][1] - landmark_neutral[i][1]])
    
#     svm_testing_data.append(ChangeVector(data_facs, landmarkChange, gt_emotion))   
    svm_testing_data.append(ImageData(data_facs, landmark, gt_emotion))
    count = count + 2


Found 1034 assets, index the returned LazyList to import.
Found 152 assets, index the returned LazyList to import.


In [6]:
#test normalize perk state
%matplotlib inline
import matplotlib.pyplot as plt


def normalize_perk_landmark(landmark_perk, landmark_neutral):
    neutral_center = landmark_neutral[30]
    perk_center = landmark_perk[30]
#     for lm in landmark_perk:
    #move perk landmark to match center of neutral landmark
    move_vector = [neutral_center[0] - perk_center[0], neutral_center[1] - perk_center[1]]
    print(move_vector)
    for lm in landmark_perk:
        lm[0] += move_vector[0]
        lm[1] += move_vector[1]
    
    print(landmark_neutral[30])
    print(landmark_perk[30])
    #scale the size of detected perk landmark based on neutrol landmark 27 to 30 (the nose)
    scale_neutral = [landmark_neutral[30][0] - landmark_neutral[27][0], landmark_neutral[30][1] - landmark_neutral[27][1]]
    scale_perk = [landmark_perk[30][0] - landmark_perk[27][0], landmark_perk[30][1] - landmark_perk[27][1]]
    print("scale_neutral:" + str(scale_neutral))
    print("scale_perk:" + str(scale_perk))
    ratio = math.sqrt(scale_neutral[0]*scale_neutral[0] + scale_neutral[1]*scale_neutral[1])/math.sqrt(scale_perk[0]*scale_perk[0]+scale_perk[1]*scale_perk[1])    
    print(ratio)
    for lm in landmark_perk:
        lm[0] = (perk_center[0] - lm[0]) * (1 - ratio) + lm[0]
        lm[1] = (perk_center[1] - lm[1]) * (1 - ratio) + lm[1]
        
    return landmark_perk

landmark_neutral = training_images[2].landmarks['PTS'].lms.points
landmark_perk = training_images[3].landmarks['PTS'].lms.points

for i in range (27,31):
    print(landmark_perk[i])

landmark_perk = normalize_perk_landmark(landmark_perk, landmark_neutral)

print("--")
for i in range (27,31):
    print(landmark_perk[i])
# x = []
# y = []
# for i in range(0,68):
#     x.append(training_images[0].landmarks['PTS'].lms.points[i][0])
#     y.append(training_images[0].landmarks['PTS'].lms.points[i][1])
# plt.plot(x,y, "ro")
# plt.axis([0, 250, 0, 250])
# plt.show()
# for i in range (27,31):
#     print(landmark_neutral[i])
# print("--")
# for i in range (27,31):
#     print(landmark_perk[i])

[  59.52933  128.8375 ]
[  74.78126  129.00418]
[  90.04854  128.82576]
[ 105.07997  129.17517]
[-0.6835399999999936, 1.6617200000000025]
[ 104.39643  130.83689]
[ 104.39643  130.83689]
scale_neutral:[34.865980000000008, -0.0026000000000294676]
scale_perk:[45.550640000000016, 0.3376700000000028]
0.7654123439586024
--
[  69.53140787  130.57843321]
[  81.20542336  130.70601214]
[  92.89118793  130.56944727]
[ 104.39643  130.83689]


In [21]:
#create model for each action unit in facs[]
models = []
au_models_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_training_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    clf = svm.SVC(kernel='linear', decision_function_shape = 'ovr')
    clf.fit(normalize(x_training, norm='max', axis=0), y_label)
    au_models_score.append(clf.score(normalize(x_training, norm='max', axis=0), y_label))
    models.append(clf)
    


    
#evaluate trained model with test data and get score
print('#######')
print('Score: ')
au_test_score = []
for au in facs:
    x_training = []
    y_label = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             y_label.append(data.facs[str(au)])
            y_label.append(1)
        else:
            y_label.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
#     print(y_label)
    au_test_score.append(models[facs.index(au)].score(normalize(x_training, norm='max', axis=0), y_label))
print(au_test_score)
print(np.mean(au_test_score))

#######
Score: 
[0.89333333333333331, 0.92000000000000004, 0.78666666666666663, 0.77333333333333332, 0.81333333333333335, 0.81333333333333335, 0.97333333333333338, 0.94666666666666666, 0.82666666666666666, 0.90666666666666662, 1.0, 0.90666666666666662, 0.81333333333333335, 0.93333333333333335, 0.89333333333333331, 0.97333333333333338, 0.77333333333333332, 0.98666666666666669, 0.94666666666666666, 0.95999999999999996, 0.90666666666666662, 0.95999999999999996, 0.97333333333333338, 0.93333333333333335, 1.0, 1.0, 1.0, 1.0, 0.94666666666666666, 0.97333333333333338, 0.95999999999999996, 0.97333333333333338, 0.98666666666666669, 0.98666666666666669, 1.0, 1.0, 1.0, 1.0]
0.932631578947


In [6]:
print(au_models_score)
print(np.mean(au_models_score))

[0.94953271028037378, 0.98691588785046724, 0.89719626168224298, 0.95327102803738317, 0.9196261682242991, 0.90467289719626165, 0.99252336448598133, 0.96635514018691593, 0.95140186915887848, 0.9738317757009346, 0.9981308411214953, 0.9719626168224299, 0.95514018691588787, 0.95887850467289715, 0.9476635514018692, 0.98691588785046724, 0.9719626168224299, 0.9981308411214953, 0.99439252336448603, 0.96635514018691593, 0.95514018691588787, 0.96635514018691593, 0.91588785046728971, 0.98878504672897194, 0.9981308411214953, 0.99626168224299061, 0.9981308411214953, 0.99626168224299061, 0.9981308411214953, 0.95514018691588787, 0.9738317757009346, 0.98504672897196266, 0.9981308411214953, 0.9719626168224299, 1.0, 0.9981308411214953, 0.99626168224299061, 0.99626168224299061, 0.99252336448598133]
0.97244188833


In [17]:
print(clf)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [36]:
print(len(svm_testing_data))

29


In [56]:
#regresssion model
wrong_predict = 0
au_score = []
for data in svm_testing_data:
    print('#####')
    local_wrong_predict = 0
    local_accurate_predict = 0
    tmp = []
    predict = []
    
    for vector in data.landmark:
        tmp.append(vector[0])
        tmp.append(vector[1])
        
    for model in models:
        if(model.predict([tmp]) != -1):
#             predict.append([facs[models.index(model)],data.facs[str(facs[models.index(model)])] , model.predict([tmp])])
            predict.append([models.index(model), model.predict([tmp])[0]])
            #print(facs[models.index(model)])
            if(str(facs[models.index(model)]) not in data.facs):
                local_wrong_predict += 1
            else: 
                local_accurate_predict += 1
        else:
            if(str(facs[models.index(model)]) in data.facs):
                local_wrong_predict += 1
    print(predict)
#     print(local_accurate_predict)
    au_score.append(float(local_accurate_predict)/float(len(data.facs)))
    print("---")
    print(data.facs)
    wrong_predict += local_wrong_predict
    
# print(wrong_predict)
# print(sum(au_score)/float(len(au_score)))

#####
[[0, 1], [1, 1], [3, 1], [6, 1], [7, 1], [8, 1], [9, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '24': 2, '12': 2}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [8, 1], [10, 1], [13, 1], [17, 1], [21, 1], [23, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 0}
#####
[[0, 1], [1, 1], [3, 1], [7, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '12': 1}
#####
[[0, 1], [1, 1], [3, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17': 3}
#####
[[0, 1], [1, 1], [3, 1], [6, 1], [8, 1], [9, 1], [10, 1], [13, 1], [17, 1], [21, 1], [24, 1], [25, 1], [30, 1], [31, 1], [32, 1], [33, 1], [34, 1]]
---
{'14': 2, '17':

In [43]:
#get predict result
y_label = []
for au in facs:
    x_training = []
    y_label_tmp = []
    #create label array
    for data in svm_testing_data:
        if(str(au) in data.facs):
#             if(data.facs[str(au)] == 0):
#                 y_label.append(2)
#             else:
            y_label_tmp.append(data.facs[str(au)])
#             y_label.append(1)
        else:
            y_label_tmp.append(-1)
        #create training data: 1x68 array, result of PCA process
        vector = []
        for tmp in data.landmark:
            vector.append(tmp[0])
            vector.append(tmp[1])
        x_training.append(vector)
    y_label.append(y_label_tmp)
for model in models:
    print("=====")
    print(model.predict(x_training))
    print("==")
    print(y_label[models.index(model)])

=====
[-1  2  2  2  2  2  2  2  2  2 -1 -1  2  2  2  2  2  2  2  2  2  2  2  2
  2  2 -1 -1  2]
==
[-1, -1, -1, -1, -1, -1, 2, -1, 2, 2, -1, -1, 3, -1, -1, -1, -1, -1, 2, 1, -1, 2, -1, -1, 4, 4, -1, -1, 2]
=====
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
==
[-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, 1, -1, -1, -1, -1, 4, 2, -1, -1, 1]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, 1, 3, 3, 4, 3, -1, 4, 4, -1, 3, 2, -1, 2, 2, -1, 2, -1, -1, 2, 3, -1, 4, 4]
=====
[2 0 0 2 2 0 0 2 2 2 2 2 2 0 2 2 2 2 0 2 2 0 2 2 0 2 2 2 2]
==
[-1, -1, -1, -1, -1, -1, -1, 1, 4, -1, 3, -1, 3, 2, -1, -1, 3, -1, 4, -1, -1, -1, 2, -1, 4, -1, -1, 1, 3]
=====
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1]
==
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1]
=====
[5 5 5 5 5 5 5 5 5

In [3]:
#build rule-based system
#emotion class have name = str, criteria = function, facs required = [[]], caculate score of input
class Emotion:
    def __init__(self, name, facs_required, criteria):
        self.name = name
        self.facs_required = facs_required
        self.criteria = criteria
    
    def criteria(self, facs_input):
        return True
    
    def score(self,facs_input = []):
        if(self.criteria(facs_input) == True):
            max = 0
            for required in self.facs_required:
                au_count = 0
                for facs in facs_input:
                    if facs in required:
                        au_count += 1
                if au_count/float(len(required)) >= max:
                    max = au_count/float(len(required))
            return max
        else:
            return 0
    
def angry_criteria(facs_input):
    if(23 in facs_input):
        return True
    return False

def disgus_criteria(facs_input):
    if(9 in facs_input or 10 in facs_input):
        return True
    return False

def fear_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input and 3 in facs_input):
        return True
    return False

def surprise_criteria(facs_input):
    if(1 in facs_input and 2 in facs_input):
        return True
    if(5 in facs_input):
        return True
    return False

def sadness_criteria(facs_input):
    return True

def happy_criteria(facs_input):
    if(12 in facs_input):
        return True
    return False

def contempt_criteria(facs_input):
    if(14 in facs_input):
        return True
    return False

happy = Emotion('happy', [[6,12]], happy_criteria)
sadness = Emotion('sadness', [[1,4,5], [6,15], [1,4,15]], sadness_criteria)
surprise = Emotion('surprise', [[1,2,5,26]], surprise_criteria)
fear = Emotion('fear', [[1,2,4,5,7,20,26]], fear_criteria)
angry = Emotion('angry', [[4,5,7,23]], angry_criteria)
disgust = Emotion('disgust', [[9,15,16], [10,15,16]], disgus_criteria)
contempt = Emotion('contempt', [[12,14]], contempt_criteria)

emotions = [happy, sadness, surprise, fear, angry, disgust, contempt]



In [15]:
#emotion task - build training data
result = []
x_training = []
y_label = []

emotions = []
for data in svm_training_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_training.append(tmp)
        y_label.append(data.emotion)
        if(data.emotion not in emotions):
            emotions.append(data.emotion)
emotions.sort()

clf = svm.LinearSVC()
clf.fit(normalize(x_training, norm='max', axis=0), y_label)
print(clf.score(normalize(x_training, norm='max', axis=0), y_label))
# print(x_training[1])


0.996336996337


In [22]:
#emotion task - multiclass SVM

#normalize using 2 set as base
def coopNormalize(set1, set2):
    len1 = len(set1)
    len2 = len(set2)
    merged = set1 + set2
    normalized = normalize(merged, norm='max', axis=0)
    result = []
    for i in range(len1, len1 + len2):
        result.append(normalized[i])
    return result


x_testing = []
y_label = []
for data in svm_testing_data:
    if(data.emotion != -1):
        tmp = []
        for vector in data.landmark:
            tmp.append(vector[0])
            tmp.append(vector[1])
        x_testing.append(tmp)
        y_label.append(data.emotion)

print(clf.predict(coopNormalize(x_training, x_testing)))
print(y_label)
print(clf.score(coopNormalize(x_training, x_testing), y_label))

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[3, 7, 1, 5, 7, 6, 4, 1, 3, 5, 7, 6, 1, 5, 7, 1, 3, 7, 6, 1, 5, 7, 1, 4, 3, 5, 7, 3, 5, 7, 6, 1, 5, 6, 7, 4, 3, 4, 7, 1, 5, 4, 7, 3, 4, 7, 5, 6, 2, 2, 2, 2, 2, 2]
0.111111111111


In [87]:
print(clf.predict([coopNormalize(x_training, x_testing)[21]]))

[5]


In [79]:
data_normalize = x_training + x_testing
landmark = [-2.6980930928635729, -0.31857955010475081, -2.4350712086726105, -0.32509667104587692, -2.4298610239616778, -0.35955207872170547, -2.1811436683827026, -0.28568767922696026, -1.9174666553812187, -0.20775041201777356, -2.7972784737445693, 1.1049142994780965, -2.9224991625867744, 1.3277957830916023, -1.23534760148749, 0.57934014892410346, -1.225183463165763, 0.82741473408228217, -3.3194317944334557, 0.1814099208488642, -4.8558478954794566, -0.36411945838878523, -5.2758224234044633, -0.82203783382576034, -4.2510266855160239, 0.049837009428216561, -2.9651567587876286, 3.4990404561355035, -2.9068832215120324, 2.0883269243263953, -2.7085528331971034, 1.7785721806339154, -2.7003767639251492, 1.778681892849761, -4.6190237758178228, 0.11972084774628655, -1.9490443023060706, 1.407556580458774, -0.11077651536596278, 1.3682180209973325, 1.0794218391341062, 0.96527008383105795, 4.3776095950579901, -1.1732056002104656, 8.5521781483821044, -1.199075533661528, 2.846780143311662, -2.6060924446163085, 0.51302708302110034, -1.5538071429818672, -3.0490022692612229, 0.26795765211662115, -3.0565040467313978, 1.5710547321875765, 0.79108954431237066, 1.6543582482893555, 0.86348226936199524, 1.0090661697904295, 0.96883851764948759, 0.89810161923134046, 1.231480734940277, -1.3355214740307133, -8.4468236045526908, -0.4018420536284566, -6.3486402132396904, -0.6535604934111916, -4.6157119253143435, -0.19689573283642403, -5.386747635136345, 1.2457653997253999, -7.7001183372715616, 1.8722125818182178, -2.7730024729577565, 0.1157438485094815, -1.8280778448305739, -1.2550505270286436, -0.5529301349515805, -2.5138494046385347, -4.4533456970056449, -1.9497642314426429, -5.8972053670511855, -1.2075327237360938, -6.2184925722695255, -0.44737470439973492, -3.7184704204775016, 3.7488370556929738, 0.058742349601686783, 5.4159598619234259, -0.82098315798390331, 4.5569615577092861, -3.0057697290332186, 4.0877720165825053, -6.8405786883978976, 3.3150975405698375, -7.6759418092206317, 3.5538767153158233, -6.2551285551713818, 0.53728256296038523, -8.7567126293202762, -2.1339225464851239, -6.1233027881500277, -0.6247115750235821, -5.2582065938950677, -0.57964158684944778, -4.5458511553240157, -0.27849336784350953, -6.9494726684200998, 0.62455353456283547, -5.1592124896214102, -1.4037405707540529, -4.5364659163077761, 0.10915270188979775, -4.4167421622358063, 0.39365236508919565, -5.3943783120719502, 0.57348173934451552, -5.8927703205571333, 0.29622284461811432, -5.7403598183086615, 0.061724813771192544, -5.1225942885987479, 0.42635986120272662, -5.1167896441550909, 0.31543715944505379, -5.5286969778268542, -0.20170118247034452, -4.7883353208642063, -0.68784555820707283, -4.7914262610444212, -1.0457930644805629, -4.7945313209947074, -0.5644573524716634, -5.9035109186196166, 0.17287507243844402, -5.128391838400205, 0.49177727042172137]
print(clf.predict([coopNormalize(data_normalize, [landmark])[0]]))

[3]


In [74]:
#save model
# save_object(models, 'models/au_models.pkl')
#save_object(facs, '/Programing/GR/Code/Python/models/facs.pkl')
# save_object(clf, 'models/emotion_model.pkl')
save_object(x_training + x_testing, 'models/normalize_data.pkl')

In [23]:
#create json log file
import json
log_path = '/Programing/GR/Code/Python/log/'
import time
import json
ts = int(time.time())
#jsonAdder = json.dumps()
output = {'n_train': len(training_images), 'n_test': len(testing_images), 'au_score' : au_models_score, 'facs': facs }
#print json.dumps(output, sort_keys=True, indent=4, separators=(',', ': '))
with open(log_path + 'system_log' + str(ts) + '.txt', "w+") as outfile:
    json.dump(output, outfile,sort_keys=True, indent=4, separators=(',', ': '))